In [ ]:
FILE='pdm-test.wav'
PDM_SAMPLE_RATE = 3072000  # target sample rate for PDM output
PCM_SAMPLE_RATE = 16000  # sample rate of the input PCM audio

import numpy as np
from deltasigma import synthesizeNTF, simulateDSM
from scipy.io import wavfile
from scipy.signal import resample


In [ ]:
# WARNING: May take a long time to run, around 15 minutes per 5 seconds of audio.

FILE='pdm-test.wav'
PDM_SAMPLE_RATE = 3072000  # target sample rate for PDM output
PCM_SAMPLE_RATE = 16000  # sample rate of the input PCM audio

import numpy as np
from deltasigma import synthesizeNTF, simulateDSM
from scipy.io import wavfile
from scipy.signal import resample

# Load a 16-bit PCM WAV
fs_input, pcm_input = wavfile.read(FILE)
pcm_resampled = resample(pcm_input, round(len(pcm_input) / fs_input * PDM_SAMPLE_RATE))
pcm_resampled = pcm_resampled / np.max(np.abs(pcm_resampled))  # normalize to [-1, 1]

# Design a 2nd-order modulator with 64× oversampling
OSR = 64
ntf = synthesizeNTF(2, OSR)

# Simulate the 1-bit PDM stream
pdm, _, _, _ = simulateDSM(pcm_resampled, ntf)

# Save as raw PDM
pdm.astype(np.int8).tofile('output.pdm')

print(len(pcm_input), '->', len(pdm), 'samples')

In [ ]:
h = np.ones(255, dtype=np.float32) / 255  # simple averaging filter

In [ ]:
pdm = np.fromfile('output.pdm', dtype=np.int8)
np.packbits(pdm > 0).tofile('output.1bit.pdm')
y = np.convolve(pdm, h, mode='same')
step = PDM_SAMPLE_RATE // PCM_SAMPLE_RATE
wavfile.write('output.wav', PCM_SAMPLE_RATE, y[::step].astype(np.float32))